In [7]:
import ranking
import salvar_resultados as sr
import imprimir_resultados as ir
import funcoes_st as fst

import pandas as pd
import numpy as np
import string

import time

In [ ]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_val = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino-Validação/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_val = pd.read_csv(f"Dados/Datasets/Treino-Validação/Validação/{arquivo}_validação.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_val.append(df_val)
    lista_df_teste.append(df_teste)

# Usando os modelos ST

In [ ]:
flag_st = True
if flag_st == True:

    nome_modelo, apelido_modelo = fst.setar_modelo(3)
    #tam = 51

    dir_metricas = f"Dados/Resultados/Ranqueado/ST-{apelido_modelo}"

    for nome, df_treino, df_val, df_teste in zip(arquivos, lista_df_treino, lista_df_val, lista_df_teste):

        '''df_teste = df_teste[:tam]
        df_treino = df_teste[:tam]
        df_val = df_teste[:tam]'''

        df_concat, df_matches = ranking.criar_df_teste(df_teste, "titulo_1", "titulo_2", 'titulo')
        
        inicio_tempo = time.time()
        embedding = fst.pipeline_st(df_treino, df_val, df_concat, nome_modelo)
        resultado = ranking.calcular_dis_2_vetores_cond(embedding, df_matches)

        final_tempo = time.time()

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

In [ ]:
    a

# Resultados

## ST-dbm-v1

### Geral

In [22]:
metodo = "ST-dbm-v1"
lista_resultado_dbm_v1 = ir.retornar_resultados(metodo)

#### Aleatório

##### 1:1

In [23]:
lista_resultado_dbm_v1[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,458.000000,458.000000,458.000000,458.000000,458.000000
mean,0.539301,0.956332,0.991266,5.676856,0.691099
std,0.498998,0.204579,0.093147,40.322176,0.351323
min,0.000000,0.000000,0.000000,1.000000,0.001464
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,683.000000,1.000000


##### 5:1

In [4]:
lista_resultado_dbm_v1[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,631.000000,631.000000,631.000000,631.000000,631.000000
mean,0.576862,0.941363,0.980983,9.952456,0.707652
std,0.494449,0.235131,0.136694,79.001935,0.358880
min,0.000000,0.000000,0.000000,1.000000,0.000710
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1408.000000,1.000000


#### Hard Negative

##### 1:1

In [5]:
lista_resultado_dbm_v1[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,530.000000,530.000000,530.000000,530.000000,530.000000
mean,0.586792,0.822642,0.918868,23.254717,0.670330
std,0.492875,0.382333,0.273296,98.407188,0.410969
min,0.000000,0.000000,0.000000,1.000000,0.000784
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,1275.000000,1.000000


##### 5:1

In [6]:
lista_resultado_dbm_v1[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,756.00000,756.000000,756.000000,756.000000,756.000000
mean,0.65873,0.854497,0.923280,42.830688,0.728523
std,0.47445,0.352840,0.266322,225.751291,0.393720
min,0.00000,0.000000,0.000000,1.000000,0.000350
25%,0.00000,1.000000,1.000000,1.000000,0.333333
50%,1.00000,1.000000,1.000000,1.000000,1.000000
75%,1.00000,1.000000,1.000000,3.000000,1.000000
max,1.00000,1.000000,1.000000,2857.000000,1.000000


### Por Categoria

#### Celulares

In [25]:
metodo = "ST-dbm-v1/celulares"
lista_resultado_dbm_v1_cell = ir.retornar_resultados(metodo)

In [26]:
lista_resultado_dbm_v1_cell[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.0,83.000000,83.000000
mean,0.493976,0.939759,1.0,3.192771,0.654271
std,0.503003,0.239379,0.0,4.751015,0.361394
min,0.000000,0.000000,1.0,1.000000,0.031250
25%,0.000000,1.000000,1.0,1.000000,0.333333
50%,0.000000,1.000000,1.0,2.000000,0.500000
75%,1.000000,1.000000,1.0,3.000000,1.000000
max,1.000000,1.000000,1.0,32.000000,1.000000


In [27]:
lista_resultado_dbm_v1_cell[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.545455,0.977273,0.977273,25.363636,0.696332
std,0.500783,0.149887,0.149887,164.223018,0.348824
min,0.000000,0.000000,0.000000,1.000000,0.000710
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1408.000000,1.000000


In [28]:
lista_resultado_dbm_v1_cell[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.506024,0.734940,0.843373,40.397590,0.586929
std,0.503003,0.444048,0.365658,150.861586,0.438595
min,0.000000,0.000000,0.000000,1.000000,0.000784
25%,0.000000,0.000000,1.000000,1.000000,0.087121
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,11.500000,1.000000
max,1.000000,1.000000,1.000000,1275.000000,1.000000


In [29]:
lista_resultado_dbm_v1_cell[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,0.545455,0.760331,0.867769,59.826446,0.636201
std,0.500000,0.428657,0.340151,184.565240,0.424140
min,0.000000,0.000000,0.000000,1.000000,0.000939
25%,0.000000,1.000000,1.000000,1.000000,0.111111
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,9.000000,1.000000
max,1.000000,1.000000,1.000000,1065.000000,1.000000


#### Fogões

In [30]:
metodo = "ST-dbm-v1/fogoes"
lista_resultado_dbm_v1_fogoes = ir.retornar_resultados(metodo)

In [31]:
lista_resultado_dbm_v1_fogoes[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.479730,0.979730,0.993243,3.648649,0.656874
std,0.501285,0.141402,0.082199,13.555060,0.346561
min,0.000000,0.000000,0.000000,1.000000,0.006173
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,162.000000,1.000000


In [32]:
lista_resultado_dbm_v1_fogoes[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,220.000000,220.000000,220.000000,220.000000,220.000000
mean,0.550000,0.927273,0.972727,6.318182,0.683495
std,0.498628,0.260281,0.163248,23.031617,0.369142
min,0.000000,0.000000,0.000000,1.000000,0.004505
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,222.000000,1.000000


In [33]:
lista_resultado_dbm_v1_fogoes[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.579545,0.840909,0.948864,12.863636,0.680264
std,0.495040,0.366804,0.220904,48.097563,0.396823
min,0.000000,0.000000,0.000000,1.000000,0.002083
25%,0.000000,1.000000,1.000000,1.000000,0.312500
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.250000,1.000000
max,1.000000,1.000000,1.000000,480.000000,1.000000


In [34]:
lista_resultado_dbm_v1_fogoes[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,228.000000,228.000000,228.000000,228.000000,228.000000
mean,0.679825,0.890351,0.956140,19.657895,0.755894
std,0.467570,0.313139,0.205233,116.541901,0.372092
min,0.000000,0.000000,0.000000,1.000000,0.000793
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,1261.000000,1.000000


#### Geladeiras

In [35]:
metodo = "ST-dbm-v1/geladeiras"
lista_resultado_dbm_v1_gel = ir.retornar_resultados(metodo)

In [36]:
lista_resultado_dbm_v1_gel[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,67.000000,67.000000,67.0,67.000000,67.000000
mean,0.582090,0.985075,1.0,1.940299,0.746352
std,0.496938,0.122169,0.0,2.235259,0.312626
min,0.000000,0.000000,1.0,1.000000,0.055556
25%,0.000000,1.000000,1.0,1.000000,0.500000
50%,1.000000,1.000000,1.0,1.000000,1.000000
75%,1.000000,1.000000,1.0,2.000000,1.000000
max,1.000000,1.000000,1.0,18.000000,1.000000


In [37]:
lista_resultado_dbm_v1_gel[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,96.000000,96.000000,96.0,96.000000,96.000000
mean,0.562500,0.958333,1.0,2.354167,0.720191
std,0.498682,0.200875,0.0,2.764356,0.336200
min,0.000000,0.000000,1.0,1.000000,0.062500
25%,0.000000,1.000000,1.0,1.000000,0.500000
50%,1.000000,1.000000,1.0,1.000000,1.000000
75%,1.000000,1.000000,1.0,2.000000,1.000000
max,1.000000,1.000000,1.0,16.000000,1.000000


In [38]:
lista_resultado_dbm_v1_gel[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,71.000000,71.000000,71.000000,71.000000,71.000000
mean,0.661972,0.915493,0.957746,6.042254,0.754471
std,0.476405,0.280126,0.202599,18.949886,0.361432
min,0.000000,0.000000,0.000000,1.000000,0.008130
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,123.000000,1.000000


In [39]:
lista_resultado_dbm_v1_gel[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,0.666667,0.842593,0.935185,56.509259,0.723120
std,0.473602,0.365882,0.247347,280.730739,0.406599
min,0.000000,0.000000,0.000000,1.000000,0.000460
25%,0.000000,1.000000,1.000000,1.000000,0.200000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,5.000000,1.000000
max,1.000000,1.000000,1.000000,2173.000000,1.000000


#### Notebooks

In [40]:
metodo = "ST-dbm-v1/notebooks"
lista_resultado_dbm_v1_note = ir.retornar_resultados(metodo)

In [41]:
lista_resultado_dbm_v1_note[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.659091,0.920455,0.977273,10.977273,0.754375
std,0.476731,0.272139,0.149887,72.759244,0.360056
min,0.000000,0.000000,0.000000,1.000000,0.001464
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,683.000000,1.000000


In [42]:
lista_resultado_dbm_v1_note[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,109.000000,109.000000,109.0,109.000000,109.000000
mean,0.688073,0.963303,1.0,2.357798,0.778628
std,0.465420,0.188886,0.0,3.184229,0.341847
min,0.000000,0.000000,1.0,1.000000,0.050000
25%,0.000000,1.000000,1.0,1.000000,0.500000
50%,1.000000,1.000000,1.0,1.000000,1.000000
75%,1.000000,1.000000,1.0,2.000000,1.000000
max,1.000000,1.000000,1.0,20.000000,1.000000


In [43]:
lista_resultado_dbm_v1_note[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,116.000000,116.000000,116.00000,116.000000,116.000000
mean,0.568966,0.775862,0.87931,39.965517,0.629277
std,0.497369,0.418823,0.32718,133.201379,0.440504
min,0.000000,0.000000,0.00000,1.000000,0.001122
25%,0.000000,1.000000,1.00000,1.000000,0.111111
50%,1.000000,1.000000,1.00000,1.000000,1.000000
75%,1.000000,1.000000,1.00000,9.000000,1.000000
max,1.000000,1.000000,1.00000,891.000000,1.000000


In [44]:
lista_resultado_dbm_v1_note[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,170.000000,170.000000,170.000000,170.000000,170.000000
mean,0.635294,0.823529,0.888235,64.888235,0.698819
std,0.482770,0.382346,0.316008,292.616202,0.414925
min,0.000000,0.000000,0.000000,1.000000,0.000408
25%,0.000000,1.000000,1.000000,1.000000,0.212500
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,4.750000,1.000000
max,1.000000,1.000000,1.000000,2452.000000,1.000000


#### TVs

In [45]:
metodo = "ST-dbm-v1/tvs"
lista_resultado_dbm_v1_tvs = ir.retornar_resultados(metodo)

In [46]:
lista_resultado_dbm_v1_tvs[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,72.000000,72.000000,72.000000,72.000000,72.000000
mean,0.527778,0.944444,0.986111,9.708333,0.675155
std,0.502731,0.230669,0.117851,59.049618,0.364848
min,0.000000,0.000000,0.000000,1.000000,0.001988
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,503.000000,1.000000


In [47]:
lista_resultado_dbm_v1_tvs[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,118.000000,118.000000,118.000000,118.000000,118.000000
mean,0.559322,0.906780,0.966102,18.432203,0.685367
std,0.498586,0.291981,0.181739,110.306138,0.376574
min,0.000000,0.000000,0.000000,1.000000,0.000923
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1084.000000,1.000000


In [48]:
lista_resultado_dbm_v1_tvs[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.642857,0.857143,0.952381,19.559524,0.717499
std,0.482035,0.352029,0.214238,91.059275,0.396909
min,0.000000,0.000000,0.000000,1.000000,0.001318
25%,0.000000,1.000000,1.000000,1.000000,0.312500
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.250000,1.000000
max,1.000000,1.000000,1.000000,759.000000,1.000000


In [49]:
lista_resultado_dbm_v1_tvs[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,129.000000,129.000000,129.000000,129.000000,129.000000
mean,0.751938,0.930233,0.953488,27.325581,0.810409
std,0.433572,0.255748,0.211411,251.747988,0.342503
min,0.000000,0.000000,0.000000,1.000000,0.000350
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,2857.000000,1.000000


## ST-dbm-v2

### Geral

In [12]:
metodo = "ST-dbm-v2"
lista_resultado_dbm_v2 = ir.retornar_resultados(metodo)

#### Aleatório

##### 1:1

In [13]:
lista_resultado_dbm_v2[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,458.000000,458.000000,458.000000,458.000000,458.000000
mean,0.456332,0.912664,0.969432,12.028384,0.618093
std,0.498634,0.282636,0.172331,73.784900,0.371893
min,0.000000,0.000000,0.000000,1.000000,0.000978
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,1023.000000,1.000000


##### 5:1

In [14]:
lista_resultado_dbm_v2[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,631.000000,631.000000,631.000000,631.000000,631.000000
mean,0.500792,0.931854,0.980983,8.838352,0.656156
std,0.500396,0.252196,0.136694,63.467921,0.364677
min,0.000000,0.000000,0.000000,1.000000,0.000841
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1189.000000,1.000000


#### Hard Negative

##### 1:1

In [15]:
lista_resultado_dbm_v2[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,530.000000,530.000000,530.000000,530.000000,530.000000
mean,0.541509,0.809434,0.909434,23.154717,0.636308
std,0.498745,0.393119,0.287262,85.094469,0.416088
min,0.000000,0.000000,0.000000,1.000000,0.000882
25%,0.000000,1.000000,1.000000,1.000000,0.200000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,5.000000,1.000000
max,1.000000,1.000000,1.000000,1134.000000,1.000000


##### 5:1

In [16]:
lista_resultado_dbm_v2[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,756.000000,756.000000,756.000000,756.000000,756.000000
mean,0.534392,0.780423,0.865079,58.111111,0.625551
std,0.499146,0.414234,0.341865,228.919028,0.423059
min,0.000000,0.000000,0.000000,1.000000,0.000367
25%,0.000000,1.000000,1.000000,1.000000,0.166667
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,6.000000,1.000000
max,1.000000,1.000000,1.000000,2728.000000,1.000000


### Por Categoria

#### Celulares

In [50]:
metodo = "ST-dbm-v2/celulares"
lista_resultado_dbm_v2_cell = ir.retornar_resultados(metodo)

In [51]:
lista_resultado_dbm_v2_cell[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.385542,0.855422,0.975904,13.506024,0.546938
std,0.489682,0.353813,0.154281,63.037034,0.386272
min,0.000000,0.000000,0.000000,1.000000,0.001869
25%,0.000000,1.000000,1.000000,1.000000,0.183333
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,5.500000,1.000000
max,1.000000,1.000000,1.000000,535.000000,1.000000


In [52]:
lista_resultado_dbm_v2_cell[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.375000,0.920455,0.977273,4.454545,0.585442
std,0.486897,0.272139,0.149887,10.744589,0.346769
min,0.000000,0.000000,0.000000,1.000000,0.012195
25%,0.000000,1.000000,1.000000,1.000000,0.312500
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,3.250000,1.000000
max,1.000000,1.000000,1.000000,82.000000,1.000000


In [53]:
lista_resultado_dbm_v2_cell[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.542169,0.831325,0.915663,21.975904,0.653082
std,0.501247,0.376741,0.279582,81.108475,0.402262
min,0.000000,0.000000,0.000000,1.000000,0.001560
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,641.000000,1.000000


In [54]:
lista_resultado_dbm_v2_cell[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,0.462810,0.760331,0.876033,63.983471,0.580434
std,0.500688,0.428657,0.330914,235.052228,0.419455
min,0.000000,0.000000,0.000000,1.000000,0.000561
25%,0.000000,1.000000,1.000000,1.000000,0.142857
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,7.000000,1.000000
max,1.000000,1.000000,1.000000,1784.000000,1.000000


#### Fogões

In [55]:
metodo = "ST-dbm-v2/fogoes"
lista_resultado_dbm_v2_fogoes = ir.retornar_resultados(metodo)

In [56]:
lista_resultado_dbm_v2_fogoes[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.439189,0.945946,0.979730,5.006757,0.602469
std,0.497973,0.226892,0.141402,13.927166,0.371307
min,0.000000,0.000000,0.000000,1.000000,0.008264
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,121.000000,1.000000


In [57]:
lista_resultado_dbm_v2_fogoes[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,220.000000,220.000000,220.000000,220.000000,220.000000
mean,0.490909,0.918182,0.977273,10.509091,0.646392
std,0.501057,0.274713,0.149373,63.285213,0.369470
min,0.000000,0.000000,0.000000,1.000000,0.001176
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,850.000000,1.000000


In [58]:
lista_resultado_dbm_v2_fogoes[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.505682,0.761364,0.880682,22.170455,0.593953
std,0.501394,0.427466,0.325087,61.244656,0.431235
min,0.000000,0.000000,0.000000,1.000000,0.001745
25%,0.000000,1.000000,1.000000,1.000000,0.100000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,10.000000,1.000000
max,1.000000,1.000000,1.000000,573.000000,1.000000


In [59]:
lista_resultado_dbm_v2_fogoes[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,228.000000,228.000000,228.000000,228.000000,228.000000
mean,0.508772,0.798246,0.885965,28.460526,0.615568
std,0.501023,0.402193,0.318553,96.031260,0.415503
min,0.000000,0.000000,0.000000,1.000000,0.001381
25%,0.000000,1.000000,1.000000,1.000000,0.166667
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,6.000000,1.000000
max,1.000000,1.000000,1.000000,724.000000,1.000000


#### Geladeiras

In [60]:
metodo = "ST-dbm-v2/geladeiras"
lista_resultado_dbm_v2_gel = ir.retornar_resultados(metodo)

In [61]:
lista_resultado_dbm_v2_gel[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,67.000000,67.000000,67.0,67.000000,67.000000
mean,0.477612,0.970149,1.0,2.343284,0.676398
std,0.503268,0.171460,0.0,2.513914,0.328197
min,0.000000,0.000000,1.0,1.000000,0.066667
25%,0.000000,1.000000,1.0,1.000000,0.416667
50%,0.000000,1.000000,1.0,2.000000,0.500000
75%,1.000000,1.000000,1.0,2.500000,1.000000
max,1.000000,1.000000,1.0,15.000000,1.000000


In [62]:
lista_resultado_dbm_v2_gel[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,96.000000,96.000000,96.000000,96.000000,96.000000
mean,0.458333,0.968750,0.989583,3.145833,0.645204
std,0.500876,0.174906,0.102062,6.982654,0.345998
min,0.000000,0.000000,0.000000,1.000000,0.014925
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,67.000000,1.000000


In [63]:
lista_resultado_dbm_v2_gel[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,71.000000,71.000000,71.000000,71.000000,71.000000
mean,0.521127,0.943662,0.971831,7.323944,0.676472
std,0.503109,0.232214,0.166633,31.318080,0.361172
min,0.000000,0.000000,0.000000,1.000000,0.003922
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,255.000000,1.000000


In [64]:
lista_resultado_dbm_v2_gel[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,108.000000,108.000000,108.000000,108.000000,108.000000
mean,0.481481,0.694444,0.805556,105.574074,0.566083
std,0.501986,0.462790,0.397618,323.141420,0.441884
min,0.000000,0.000000,0.000000,1.000000,0.000514
25%,0.000000,0.000000,1.000000,1.000000,0.045455
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,22.000000,1.000000
max,1.000000,1.000000,1.000000,1947.000000,1.000000


#### Notebooks

In [65]:
metodo = "ST-dbm-v2/notebooks"
lista_resultado_dbm_v2_note = ir.retornar_resultados(metodo)

In [66]:
lista_resultado_dbm_v2_note[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.500000,0.852273,0.909091,25.806818,0.629266
std,0.502865,0.356863,0.289127,111.096919,0.395753
min,0.000000,0.000000,0.000000,1.000000,0.001053
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,0.500000,1.000000,1.000000,1.500000,0.750000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,950.000000,1.000000


In [67]:
lista_resultado_dbm_v2_note[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.614679,0.926606,0.981651,7.724771,0.721513
std,0.488919,0.261987,0.134829,45.492953,0.368869
min,0.000000,0.000000,0.000000,1.000000,0.002114
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,473.000000,1.000000


In [68]:
lista_resultado_dbm_v2_note[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.534483,0.732759,0.870690,40.706897,0.600656
std,0.500974,0.444439,0.336999,133.215877,0.445988
min,0.000000,0.000000,0.000000,1.000000,0.000882
25%,0.000000,0.000000,1.000000,1.000000,0.083807
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,12.250000,1.000000
max,1.000000,1.000000,1.000000,1134.000000,1.000000


In [69]:
lista_resultado_dbm_v2_note[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,170.000000,170.000000,170.000000,170.000000,170.000000
mean,0.541176,0.747059,0.811765,84.676471,0.613066
std,0.499774,0.435982,0.392055,279.573949,0.440410
min,0.000000,0.000000,0.000000,1.000000,0.000367
25%,0.000000,0.250000,1.000000,1.000000,0.093182
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,10.750000,1.000000
max,1.000000,1.000000,1.000000,2728.000000,1.000000


#### TVs

In [70]:
metodo = "ST-dbm-v2/tvs"
lista_resultado_dbm_v2_tvs = ir.retornar_resultados(metodo)

In [71]:
lista_resultado_dbm_v2_tvs[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,72.000000,72.000000,72.000000,72.000000,72.000000
mean,0.500000,0.930556,0.986111,16.930556,0.664320
std,0.503509,0.255992,0.117851,120.283389,0.358392
min,0.000000,0.000000,0.000000,1.000000,0.000978
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,0.500000,1.000000,1.000000,1.500000,0.750000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1023.000000,1.000000


In [72]:
lista_resultado_dbm_v2_tvs[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,118.000000,118.000000,118.000000,118.000000,118.000000
mean,0.542373,0.940678,0.983051,14.652542,0.675632
std,0.500326,0.237234,0.129631,109.891895,0.374117
min,0.000000,0.000000,0.000000,1.000000,0.000841
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1189.000000,1.000000


In [73]:
lista_resultado_dbm_v2_tvs[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.642857,0.880952,0.964286,15.523810,0.723764
std,0.482035,0.325790,0.186691,74.966897,0.388450
min,0.000000,0.000000,0.000000,1.000000,0.001567
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,638.000000,1.000000


In [74]:
lista_resultado_dbm_v2_tvs[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,129.000000,129.000000,129.000000,129.000000,129.000000
mean,0.682171,0.883721,0.937984,30.263566,0.751756
std,0.467448,0.321809,0.242124,212.709548,0.379368
min,0.000000,0.000000,0.000000,1.000000,0.000430
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,2324.000000,1.000000


## ST-pml-v2

### Geral

In [17]:
metodo = "ST-pml-v2"
lista_resultado_pml_v2 = ir.retornar_resultados(metodo)

#### Aleatório

##### 1:1

In [18]:
lista_resultado_pml_v2[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,458.000000,458.000000,458.000000,458.000000,458.000000
mean,0.530568,0.927948,0.993450,5.449782,0.669691
std,0.499610,0.258857,0.080756,29.944491,0.368880
min,0.000000,0.000000,0.000000,1.000000,0.001848
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,541.000000,1.000000


##### 5:1

In [19]:
lista_resultado_pml_v2[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,631.000000,631.000000,631.000000,631.000000,631.000000
mean,0.534073,0.957211,0.985737,11.782884,0.684665
std,0.499233,0.202542,0.118667,139.825545,0.355226
min,0.000000,0.000000,0.000000,1.000000,0.000310
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,3227.000000,1.000000


#### Hard Negative

##### 1:1

In [20]:
lista_resultado_pml_v2[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,530.000000,530.000000,530.000000,530.000000,530.000000
mean,0.626415,0.828302,0.937736,13.558491,0.700678
std,0.484212,0.377474,0.241863,47.586250,0.404252
min,0.000000,0.000000,0.000000,1.000000,0.001905
25%,0.000000,1.000000,1.000000,1.000000,0.270833
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.750000,1.000000
max,1.000000,1.000000,1.000000,525.000000,1.000000


##### 5:1

In [21]:
lista_resultado_pml_v2[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,756.000000,756.000000,756.000000,756.000000,756.000000
mean,0.689153,0.873016,0.923280,39.973545,0.753158
std,0.463147,0.333176,0.266322,201.028546,0.382148
min,0.000000,0.000000,0.000000,1.000000,0.000438
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,2282.000000,1.000000


### Por Categoria

#### Celulares

In [75]:
metodo = "ST-pml-v2/celulares"
lista_resultado_pml_v2_cell = ir.retornar_resultados(metodo)

In [76]:
lista_resultado_pml_v2_cell[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.0,83.000000,83.000000
mean,0.530120,0.975904,1.0,2.493976,0.675727
std,0.502126,0.154281,0.0,2.471168,0.358279
min,0.000000,0.000000,1.0,1.000000,0.066667
25%,0.000000,1.000000,1.0,1.000000,0.333333
50%,1.000000,1.000000,1.0,1.000000,1.000000
75%,1.000000,1.000000,1.0,3.000000,1.000000
max,1.000000,1.000000,1.0,15.000000,1.000000


In [77]:
lista_resultado_pml_v2_cell[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.443182,0.954545,0.977273,42.261364,0.628972
std,0.499608,0.209493,0.149887,344.498019,0.353866
min,0.000000,0.000000,0.000000,1.000000,0.000310
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,3227.000000,1.000000


In [78]:
lista_resultado_pml_v2_cell[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.710843,0.855422,0.951807,9.662651,0.772253
std,0.456127,0.353813,0.215475,33.531302,0.375205
min,0.000000,0.000000,0.000000,1.000000,0.003571
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,280.000000,1.000000


In [79]:
lista_resultado_pml_v2_cell[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,0.586777,0.826446,0.884298,64.322314,0.677571
std,0.494460,0.380300,0.321198,245.335050,0.406348
min,0.000000,0.000000,0.000000,1.000000,0.000644
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1554.000000,1.000000


#### Fogões

In [80]:
metodo = "ST-pml-v2/fogoes"
lista_resultado_pml_v2_fogoes = ir.retornar_resultados(metodo)

In [81]:
lista_resultado_pml_v2_fogoes[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.513514,0.891892,0.986486,6.445946,0.654741
std,0.501515,0.311571,0.115852,27.695694,0.375506
min,0.000000,0.000000,0.000000,1.000000,0.003802
25%,0.000000,1.000000,1.000000,1.000000,0.250000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,4.000000,1.000000
max,1.000000,1.000000,1.000000,263.000000,1.000000


In [82]:
lista_resultado_pml_v2_fogoes[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,220.000000,220.000000,220.000000,220.000000,220.000000
mean,0.468182,0.940909,0.986364,7.081818,0.636658
std,0.500125,0.236333,0.116240,43.197567,0.359995
min,0.000000,0.000000,0.000000,1.000000,0.001684
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,0.000000,1.000000,1.000000,2.000000,0.500000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,594.000000,1.000000


In [83]:
lista_resultado_pml_v2_fogoes[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.562500,0.818182,0.960227,13.420455,0.645655
std,0.497494,0.386795,0.195982,50.216496,0.418399
min,0.000000,0.000000,0.000000,1.000000,0.001905
25%,0.000000,1.000000,1.000000,1.000000,0.166667
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,6.000000,1.000000
max,1.000000,1.000000,1.000000,525.000000,1.000000


In [84]:
lista_resultado_pml_v2_fogoes[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,228.000000,228.000000,228.000000,228.000000,228.000000
mean,0.666667,0.877193,0.951754,18.153509,0.736430
std,0.472442,0.328938,0.214756,137.340947,0.386144
min,0.000000,0.000000,0.000000,1.000000,0.000505
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1979.000000,1.000000


#### Geladeiras

In [85]:
metodo = "ST-pml-v2/geladeiras"
lista_resultado_pml_v2_gel = ir.retornar_resultados(metodo)

In [86]:
lista_resultado_pml_v2_gel[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,67.000000,67.000000,67.0,67.000000,67.000000
mean,0.567164,0.970149,1.0,2.388060,0.735700
std,0.499208,0.171460,0.0,4.417442,0.318882
min,0.000000,0.000000,1.0,1.000000,0.029412
25%,0.000000,1.000000,1.0,1.000000,0.500000
50%,1.000000,1.000000,1.0,1.000000,1.000000
75%,1.000000,1.000000,1.0,2.000000,1.000000
max,1.000000,1.000000,1.0,34.000000,1.000000


In [87]:
lista_resultado_pml_v2_gel[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,96.000000,96.000000,96.0,96.000000,96.000000
mean,0.479167,0.979167,1.0,2.395833,0.667410
std,0.502188,0.143576,0.0,2.332738,0.337962
min,0.000000,0.000000,1.0,1.000000,0.071429
25%,0.000000,1.000000,1.0,1.000000,0.333333
50%,0.000000,1.000000,1.0,2.000000,0.500000
75%,1.000000,1.000000,1.0,3.000000,1.000000
max,1.000000,1.000000,1.0,14.000000,1.000000


In [88]:
lista_resultado_pml_v2_gel[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,71.000000,71.000000,71.000000,71.000000,71.000000
mean,0.661972,0.957746,0.985915,3.478873,0.769461
std,0.476405,0.202599,0.118678,9.641961,0.340864
min,0.000000,0.000000,0.000000,1.000000,0.013333
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,75.000000,1.000000


In [89]:
lista_resultado_pml_v2_gel[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,108.000000,108.000000,108.000000,108.00000,108.000000
mean,0.648148,0.870370,0.898148,53.37037,0.729918
std,0.479774,0.337461,0.303863,262.90923,0.387822
min,0.000000,0.000000,0.000000,1.00000,0.000438
25%,0.000000,1.000000,1.000000,1.00000,0.500000
50%,1.000000,1.000000,1.000000,1.00000,1.000000
75%,1.000000,1.000000,1.000000,2.00000,1.000000
max,1.000000,1.000000,1.000000,2282.00000,1.000000


#### Notebooks

In [90]:
metodo = "ST-pml-v2/notebooks"
lista_resultado_pml_v2_note = ir.retornar_resultados(metodo)

In [91]:
lista_resultado_pml_v2_note[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,88.000000,88.000000,88.0,88.000000,88.000000
mean,0.465909,0.886364,1.0,5.090909,0.592038
std,0.501695,0.319188,0.0,7.594064,0.400433
min,0.000000,0.000000,1.0,1.000000,0.031250
25%,0.000000,1.000000,1.0,1.000000,0.191667
50%,0.000000,1.000000,1.0,2.000000,0.500000
75%,1.000000,1.000000,1.0,5.250000,1.000000
max,1.000000,1.000000,1.0,32.000000,1.000000


In [92]:
lista_resultado_pml_v2_note[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.642202,0.935780,0.972477,4.431193,0.737191
std,0.481566,0.246277,0.164357,10.918356,0.369577
min,0.000000,0.000000,0.000000,1.000000,0.014286
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,70.000000,1.000000


In [93]:
lista_resultado_pml_v2_note[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.500000,0.672414,0.836207,29.310345,0.574544
std,0.502169,0.471369,0.371693,71.869134,0.446545
min,0.000000,0.000000,0.000000,1.000000,0.002137
25%,0.000000,0.000000,1.000000,1.000000,0.046584
50%,0.500000,1.000000,1.000000,1.500000,0.750000
75%,1.000000,1.000000,1.000000,21.500000,1.000000
max,1.000000,1.000000,1.000000,468.000000,1.000000


In [94]:
lista_resultado_pml_v2_note[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,170.000000,170.000000,170.000000,170.000000,170.000000
mean,0.664706,0.841176,0.894118,70.247059,0.725893
std,0.473488,0.366591,0.308596,255.712852,0.402132
min,0.000000,0.000000,0.000000,1.000000,0.000571
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1752.000000,1.000000


#### TVs

In [95]:
metodo = "ST-pml-v2/tvs"
lista_resultado_pml_v2_tvs = ir.retornar_resultados(metodo)

In [96]:
lista_resultado_pml_v2_tvs[0][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,72.000000,72.000000,72.000000,72.000000,72.000000
mean,0.611111,0.958333,0.986111,10.097222,0.726945
std,0.490919,0.201229,0.117851,63.595367,0.359488
min,0.000000,0.000000,0.000000,1.000000,0.001848
25%,0.000000,1.000000,1.000000,1.000000,0.333333
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,541.000000,1.000000


In [97]:
lista_resultado_pml_v2_tvs[1][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,118.000000,118.000000,118.000000,118.000000,118.000000
mean,0.669492,0.991525,0.991525,12.245763,0.781222
std,0.472402,0.092057,0.092057,112.057139,0.325672
min,0.000000,0.000000,0.000000,1.000000,0.000820
25%,0.000000,1.000000,1.000000,1.000000,0.500000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,2.000000,1.000000
max,1.000000,1.000000,1.000000,1219.000000,1.000000


In [98]:
lista_resultado_pml_v2_tvs[2][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.821429,0.928571,0.976190,4.464286,0.861290
std,0.385293,0.259086,0.153371,15.403369,0.308495
min,0.000000,0.000000,0.000000,1.000000,0.008475
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,118.000000,1.000000


In [99]:
lista_resultado_pml_v2_tvs[3][["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe()

,k-1,k-10,k-50,match_rank,1/match_rank
count,129.000000,129.000000,129.000000,129.000000,129.000000
mean,0.891473,0.953488,0.968992,4.589147,0.909013
std,0.312258,0.211411,0.174014,18.369307,0.266647
min,0.000000,0.000000,0.000000,1.000000,0.006098
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,164.000000,1.000000
